# Multi Link Redundancy

The multi-link redundancy analysis returns the same types of results as the single-link redundancy analysis, but it considers multiple links failing simultaneously. This is particularly useful for assessing the resilience of a network to natural disasters that may affect several links at once.

## Run Multi-Link Redundancy Analysis

The workflow is therefore very similar to the single-link redundancy analysis, with the difference being that a hazard map must be specified. See the [Single Link Redundancy](../tutorials/criticality_single_link_redundancy.html) tutorial for more details on the workflow.

We select the analysis type as [AnalysisLossesEnum.MULTI_LINK_REDUNDANCY](../api/ra2ce.analysis.analysis_config_data.enums.analysis_losses_enum.html#ra2ce.analysis.analysis_config_data.enums.analysis_losses_enum.AnalysisLossesEnum.MULTI_LINK_REDUNDANCY){.api-ref}, for which we can also set a threshold for the water depth. The threshold defines the hazard value above which a link is considered impassable/disrupted.

## Step 1: Import Libraries and Set Paths

We start by importing the required libraries and defining the root directory and network path.

In [1]:
from pathlib import Path
import geopandas as gpd
from IPython.core.display_functions import display

from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisSectionLosses, AnalysisConfigData
from ra2ce.analysis.analysis_config_data.enums.analysis_losses_enum import AnalysisLossesEnum
from ra2ce.analysis.analysis_config_data.enums.weighing_enum import WeighingEnum
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.network_config_data import NetworkSection, NetworkConfigData, HazardSection
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.ra2ce_handler import Ra2ceHandler

root_dir = Path('data', 'multi_link_redundancy')

network_path = root_dir / "network"
hazard_path = root_dir / "hazard"


c:\Users\hauth\miniforge3\envs\ra2ce_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 2: Define Network and Analysis Configuration

In [3]:
network_section = NetworkSection(
    source=SourceEnum.SHAPEFILE,
    primary_file=network_path.joinpath("base_shapefile.shp"),
    file_id="ID",
    save_gpkg=True)

hazard_section = HazardSection(
    hazard_map=[hazard_path.joinpath("max_flood_depth.tif")],
    hazard_id='Flood',
    hazard_field_name="waterdepthtt",
    aggregate_wl=AggregateWlEnum.MIN,
    hazard_crs="EPSG:32736",
    )

network_config_data = NetworkConfigData(
    root_path= root_dir,
    static_path=root_dir.joinpath('static'),
    network=network_section,
    hazard=hazard_section,
    )

analyse_section = AnalysisSectionLosses(
    name="tutorial_multi_link_redundancy",
    analysis=AnalysisLossesEnum.MULTI_LINK_REDUNDANCY,
    threshold=0.3,  # roads with a flood depth above this value are considered impassable
    weighing=WeighingEnum.LENGTH,
    save_csv=True,
    save_gpkg=True,
)

analysis_config_data = AnalysisConfigData(
    output_path=root_dir.joinpath("output"),
    static_path=root_dir.joinpath('static'),
    analyses=[analyse_section],
    aggregate_wl=AggregateWlEnum.MIN,
)

handler = Ra2ceHandler.from_config(network=network_config_data, analysis=analysis_config_data)
handler.configure()
handler.run_analysis()

100%|██████████| 4217/4217 [00:00<00:00, 370052.09it/s]
2025-10-01 05:04:27 PM - [avg_speed_calculator.py:175] - root - WARNING - No valid file found with average speeds data\multi_link_redundancy\static\output_graph\avg_speed.csv, calculating and saving them instead.
2025-10-01 05:04:27 PM - [avg_speed_calculator.py:175] - root - WARNING - No valid file found with average speeds data\multi_link_redundancy\static\output_graph\avg_speed.csv, calculating and saving them instead.
2025-10-01 05:04:27 PM - [avg_speed_calculator.py:150] - root - WARNING - Default speed have been assigned to road type [<RoadTypeEnum.SECONDARY_LINK: 8>]. Please check the average speed CSV, enter the right average speed for this road type and run RA2CE again.
2025-10-01 05:04:27 PM - [avg_speed_calculator.py:150] - root - WARNING - Default speed have been assigned to road type [<RoadTypeEnum.SECONDARY_LINK: 8>]. Please check the average speed CSV, enter the right average speed for this road type and run RA2CE a

[AnalysisResultWrapper(results_collection=[AnalysisResult(analysis_result=         u     v link_id    ID      highway  avgspeed  \
 0        0     1    None  None     tertiary      60.0   
 1        0     2    None  None     tertiary      58.0   
 2        0     3    None  None     tertiary      58.0   
 3        1   465    None  None     tertiary      60.0   
 4        1   663    None  None  residential      60.0   
 ...    ...   ...     ...   ...          ...       ...   
 4213  1481  1482    None  None  residential      60.0   
 4214  1481  1483    None  None  residential      60.0   
 4215  1481  1484    None  None  residential      60.0   
 4216  1508  1512    None  None  residential      60.0   
 4217  1513  1514    None  None  residential      60.0   
 
                                                geometry lanes  length_x  \
 0     LINESTRING (34.87673 -19.85047, 34.87737 -19.8...   nan      70.0   
 1     LINESTRING (34.87673 -19.85047, 34.87642 -19.8...   nan      70.0   
 

## Inspect Results

In [ ]:
analysis_output_folder = root_dir.joinpath("output", "multi_link_redundancy")
redundancy_gdf = gpd.read_file(analysis_output_folder/"tutorial_multi_link_redundancy.gpkg") # specify the name of the geopackage holding your results (can be found in the analysis output folder)
redundancy_gdf.head() # display the attributes of the file

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))
redundancy_gdf.plot(column='alt_length', ax=ax, legend=False, cmap='viridis')
plt.title('Multi Link Redundancy Analysis Results')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.show()